In [1]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
import os

from google.cloud import storage

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import regularizers

import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots

from numpy import savez_compressed
from numpy import load

# Config

In [2]:
split_path = 'data/split'

In [3]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'ServiceKey_GoogleCloud.json'
storage_client = storage.Client()
data_bucket = storage_client.get_bucket('exsem-dog-classification')

In [4]:
def upload_to_bucket(file_path, bucket_name):
    try:
        bucket = storage_client.get_bucket(bucket_name)
        blob = bucket.blob(file_path)
        blob.upload_from_filename(file_path)
        return 'succesfull upload'
    except Exception as e:
        print(e)
        return 'upload failed'

In [5]:
def download_file_from_bucket(file_path, bucket_name):
    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(file_path)
    with open(file_path, 'wb') as f:
        storage_client.download_blob_to_file(blob, f)
    print('Saved')

# Load & Prep Data

In [ ]:
download_file_from_bucket(split_path + '/X_train.npz', data_bucket)
download_file_from_bucket(split_path + '/X_test.npz', data_bucket)
download_file_from_bucket(split_path + '/X_val.npz', data_bucket)
download_file_from_bucket(split_path + '/y_train.npy', data_bucket)
download_file_from_bucket(split_path + '/y_test.npy', data_bucket)
download_file_from_bucket(split_path + '/y_val.npy', data_bucket)

In [6]:
X_train = load(split_path + '/X_train.npz')['arr_0']
X_test = load(split_path + '/X_test.npz')['arr_0']
X_val = load(split_path + '/X_val.npz')['arr_0']

y_train = np.load(split_path + '/y_train.npy')
y_test = np.load(split_path + '/y_test.npy')
y_val = np.load(split_path + '/y_val.npy')

In [ ]:
X_train = X_train / np.max(X_train)
X_test = X_test / np.max(X_test)
X_val = X_val / np.max(X_val)

# Model building

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters= 32, kernel_size= (3,3), activation='relu', input_shape = (255, 255,3)),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(120, activation='softmax')
])

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy', 'sparse_categorical_crossentropy']) 

In [ ]:
model.fit(